# Урок 3. Парсинг данных. HTML, Beautiful Soap


1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля)

In [1]:
import certifi

In [2]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import json

In [3]:
pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 30.1 MB/s eta 0:00:00


In [4]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

In [5]:
client = MongoClient('127.0.0.1', 27017)
db = client['HH_ru']
jobs = db.jobs

In [6]:
base_url = 'https://murmansk.hh.ru/'
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
           "Accept": "*/*"}
url = f'{base_url}/search/vacancy'

In [7]:
vacancy_list = []

In [8]:
zp_min = int(input('Zp min: '))

Zp min: 73000


In [11]:
for i in range(1):

  params = {'clusters' : 'true', 'area' : 1, 'ored_clusters' : 'true', 'enable_snippets' : 'true', 'salary' : '', 'st':'searchVacancy','text': 'pharma'}
  params['page'] = int(i)
  response = requests.get(url, headers=headers, params=params)
  dom = BeautifulSoup(response.text, 'html.parser')

  vacancy = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})

  for items in vacancy:
    vacancy_data = {}

    info = items.find('a', {'class': 'serp-item__title'})
    link = base_url + info['href']
    name = info.getText()
    company = items.find('a', {'class': 'bloko-link_kind-tertiary'}).getText()
    views = items.find('div', {'class': 'online-users'})
  
    try:
      salary = items.find('span', {'class': 'bloko-header-section-3'})
  
      if len(salary.split()) == 6:
        min_salary = salary.split()[0] + salary.split()[1]
        max_salary = salary.split()[3] + salary.split()[4]
        valute = salary.split()[5]

      elif len(salary.split()) == 4 and salary.split()[0] == 'от':
        min_salary = salary.split()[1] + salary.split()[2]
        max_salary = None
        valute = salary.split()[3]


      else:
        min_salary = None
        max_salary = salary.split()[1] + salary.split()[2]
        valute = salary.split()[3]
    except:
      min_salary = None
      max_salary = None
      valute = None

    vacancy_data['name'] = name
    vacancy_data['link'] = link
    vacancy_data['company'] = company
    vacancy_data['views'] = views
    try:
      vacancy_data['min_salary'] = int(min_salary)
      vacancy_data['max_salary'] = int(max_salary)
      vacancy_data['valute'] = valute
    except:
      vacancy_data['min_salary'] = min_salary
      vacancy_data['max_salary'] = max_salary
      vacancy_data['valute'] = valute
    
    try:
      if vacancy_data['min_salary'] >= zp_min or vacancy_data['max_salary'] >= zp_min:
        pprint(vacancy_data)
        print('\n')
    except:
      pass

    jobs.update_one({'link': vacancy_data['link']}, {'$set': vacancy_data}, upsert=True)


f = 0
for i in jobs.find({}):
  f += 1
